## 本教程通过使用溶度积与相对分子质量计算溴酸银在不同溶液中的溶解度，本教程需要且仅需要拥有一定相应的高中化学背景知识以及一定的python基础。



# 背景知识 1&2

#### 1. 活度， 通常我们计算各种溶液时，会使用浓度来描述单位体积内物质的量的数量，而在各类热力学计算过程中，浓度也与各类热力学系数相关。然而，当我们应用于真实溶液时，会发现溶液内的真实浓度的计算结果存在偏差。为了修正这种偏差，提出了活度与活度系数作为修正。$$a_i = \gamma_{i} c_{i}/c^{\theta} \tag{1}$$其中$a_i$是物质$i$的活度，$\gamma_i$是物质$i$的活度系数,$c_i$是物质$i$的真实浓度,$c^{\theta}$是标准浓度$1mol/L$。对于纯水溶液，我们一般认为活度系数为1.

#### 2. 溴酸银$AgBrO_{3}$ 是一种难溶于水的无机盐，通常我们讨论的溶解度对应的电离方程式如下:$$AgBrO_{3}(s) \rightleftharpoons Ag^{+}(aq) +BrO_{3}^{-}(aq)\tag{2}$$对于$AgBrO_3$的溶解过程,我们定义活度积$$K_{ap,AgBrO_{3}} = a(Ag^{+})a(BrO_{3}^{-})\tag{3}$$

#### 通过查阅资料，我们可以了解到，溴酸银的活度积数值为$K_{ap,AgBrO_{3}} =5.77E-5\tag{4}$

#### 3. 溶解度S，我们可以将其简单定义为单位体积内溶解物质的质量，值得注意的是，在化学中，这个单位通常可以是$g/L$。$$S_{i} = \frac{m_{i}}{V_{i}} \tag{5}$$ 在这里$m_i$是物质$i$溶解的质量，$V_i$是溶解物质$i$的溶剂的体积。


#### 好像已经有充足的背景知识，现在我们可以试着计算$AgBrO_3$在水中的溶解度了!

In [81]:
# 这个代码块用于引用库,但是现在需要引用什么库呢？
import math as ma
import numpy as np
from scipy.optimize import fsolve

#### 首先，借助于[式(5)],我们知道可以通过检查一定体积的$AgBrO_3$饱和水溶液中，溶解的$AgBrO_3$质量来得到溶解度，可惜的是，我们现在没有这样做的条件，如何通过我们已知的量[式(4)]来得到溶解度呢？

#### 或许你应该知道，在开头我们提到过，浓度恰好也是一种描述“单位体积溶剂中物质的量”的物理量，如果我们认为溶剂和溶液的体积在溶解物质前后没有发生变化的话。那么，我们就可以写出公式$$c_i = \frac{n_{i}}{V_i} \tag{6}$$其中$n_{i}$是物质$i$的物质的量。

#### 这是你应该发现了，有一种方法可以将物质的量和质量联系在一起的方法，那就是相对分子质量$M_{i}$,单位是$g/mol$。那么我们就有$$m_{i} = M_{i}n_{i} \tag{7}$$
#### 在这里，我们给出$AgBrO_3$的相对分子质量$M_{AgBrO_3} = 235.8 g/mol \tag{8}$

#### 那么我们只需要将公式(5),(6),(7)结合在一起，得到$$S_{i} = \frac{m_{i}}{V_{i}} = \frac{m_{i}c_{i}}{n_{i}} = M_{i}c_{i} \tag{9} $$ 其中$M_{i}$对应的$M_{AgBrO_{3}}$ 是公式(8)中给出的已知量。值得注意是，这里我们使用了一个先决条件，即使用了饱和溶液的浓度，公式中的$c_{i}$指代的应是饱和溶液中物质$i$的浓度。

#### 我们可以先定义一些函数和已知量，将我们的推导记录一下。

In [82]:
Kap_AgBrO3 = 5.77E-5
molecular_weight_AgBrO3 = 235.8

def solubility(mol_weight,concentration):
    return mol_weight*concentration


#### 我们定义了一个solubility函数，通过传入相对分子质量mol_weight和饱和浓度concentration来计算溶解度。
#### 而通过上文，我们也已经定义了$AgBrO_{3}$的相对分子质量,那么我们下一步应该将公式中的浓度替换为活度。在python中，我们需要定义一个通过活度a计算浓度c的函数,这个函数基于公式(1)。

In [83]:
def trans_activ_to_conc(activity_coeffi,activity):
    return activity/activity_coeffi

#### 现在我们面临了一个新的问题，溶解在水溶液中的$AgBrO_{3}$的浓度，和我们给出公式(3)中的$ a(Ag^{+})a(BrO_{3}^{-})$有什么联系呢？
#### 在水溶液中，我们可以判断出来，溶解在水溶液中的$AgBrO_{3}$,作为一种强电解质，由于电离的组分在水溶液中不会（或者在本次讨论中不会）转化为别的物质或消失或凭空出现。它的浓度应该等于溴酸银电离组分的浓度。$$c_{AgBrO_{3}} = c_{Ag^{+}} = c_{BrO_{3}^{-}}\tag{10}$$那么，结合公式(10),(4).$$K_{ap,AgBrO_{3}} = a(Ag^{+})a(BrO_{3}^{-}) = a^{2}(AgBrO_{3}) \tag{11}$$即$$a(AgBrO_3) = \sqrt{K_{ap,AgBrO_{3}}}\tag{12}$$

#### 至此，我们只需要简单定义一个变量，将其引入函数中即可。

In [84]:
activity_AgBrO3 = Kap_AgBrO3**0.5

solubility_AgBrO3_in_water  = solubility(molecular_weight_AgBrO3,trans_activ_to_conc(1,activity_AgBrO3))

print(str(solubility_AgBrO3_in_water)+"g/L") 

1.7911489686790434g/L


#### 下面一个问题可能更有难度一些，0.01mol/L的$KBrO_{3}$的水溶液中，$AgBrO_{3}$的溶解度会是多少？
#### 要解决的首要问题是，由于这次不是纯水溶液，因此活度系数会有所不同，因此我们这里要引出一个计算理论活度系数的公式--德拜-休克尔极限公式。$$lg\gamma = A |z_{+}z_{-}|\sqrt{I}\tag{13}$$其中A在一定温度下水溶液中为常数,在本问题中$A = 0.51 $,$z_{+}z_{-}$分别为溶液中正负离子的电荷，$I$为离子强度。对于带一个电荷的粒子，离子强度I是所有粒子浓度的和的二分之一。
$$I = \frac{1}{2}\sum_{i}c_{i}\tag{14}$$ 

#### 对于本问题，我们可以将公式(13),(14)结合得到
$$lg\gamma = -0.51*|1*1| *\sqrt{0.5*(c_{K^{+}}+c_{Ag^{+}}+c_{BrO_{3}^{-}}}) = -0.51\sqrt{0.5*(c_{K^{+}}+c_{Ag^{+}}+c_{BrO_{3}^{-}})}\tag{15}$$
#### 我们可以定义函数

In [85]:
def calc_gamma(list_of_conc):
    
    conc_sum = sum(list_of_conc)
    return 10**(-0.509*(0.5*conc_sum)**0.5)


#### 现在我们好像遇到了一些小小的问题，不是吗？如果你仔细观察，你会发现为了求出gamma，我们需要溶液中所有离子的浓度，为了求出所有粒子的浓度，我们需要知道$AgBrO_3$的溶解度，为了知道溶解度，我们又需要gamma。这样一来，我们根本没有办法解决这样的问题！

#### 所幸的是我们有不同的方法来解决这个问题，首先，我们可以试着使用迭代的方法求得数值解。迭代解的思路在于，为了进行求解gamma-浓度-溶解度的循环，我们需要先“猜”一个数据，让这个循环不断的“迭代”下去，通常而言，这样的迭代最终会收敛于一组数据，而这就是我们想要的数值解。
#### 为了求解，我们需要定义一个“收敛限”，并猜出一个数据，不过好消息是，在这个循环中，有一个数据在之前的问题中已经得到了不错的近似值，那就是溶解度，我们可以先将纯水中的溶解度代入这个循环来启动它。
#### 在正式迭代之前，我们还要解决一个问题，在纯水中，我们可以简单的得到公式(12) $$a(AgBrO_3) = \sqrt{K_{ap,AgBrO_{3}}}\tag{12}$$然而在这里，由于加入了$KBrO_{3}$,我们只能通过假定的gamma求解对应的活度。因此，我们还要定义一个通过gamma求解活度的程序

In [105]:

solubility_list = [solubility_AgBrO3_in_water] 
solu_mol_AgBrO3_in_water = solubility_AgBrO3_in_water/molecular_weight_AgBrO3 #在这里，记得我们要把溶解度转化为溶解的物质的量!
conc_list = [0.01,solu_mol_AgBrO3_in_water,0.01+solu_mol_AgBrO3_in_water]
def calc_activ(Kap,tem_gamma,conc_list):
    b = conc_list[0]
    a = 1
    c = -Kap/(tem_gamma**2)
    return (-b+(b**2-4*a*c)**0.5)/2
for i in range(100): #通常对于这些简单方程，100次以内的迭代是充足的
    tem_gamma = calc_gamma(conc_list)
    tem_conc = calc_activ(Kap_AgBrO3,tem_gamma,conc_list)
    conc_list = [0.01,tem_conc,0.01+tem_conc]
    solubility_list.append(solubility(molecular_weight_AgBrO3,tem_conc))
    #print(tem_conc)
    if abs(solubility_list[-1]-solubility_list[-2]) < 0.0001: #根据我们预估的数值，0.0001的收敛限制应该属足够的
        break
        
print("溶解度为"+str(solubility_list[-1]))
    

溶解度为1.201985308843194


#### 另一种方法是使用scipy库中的fsolve功能，通过定义定义函数 func,fsolve功能会根据给定的初始值求解函数的零点。事实上，如果我们列出一个方程，并将所有项归到同侧，将其定义为一个函数，那么求该函数的零点就等于求解该方程。
#### 通常而言，这类直接求解方程的要更加直观快速，但缺点则是相对于迭代的方法，使用者每次都要自行列出并输入方程。

#### 在这个问题中，我们需要假设溶解的$AgBrO_3$的浓度为x，那么此时就有$$c_{Ag^{+}} = x  \tag{16}$$
$$c_{BrO_{3}^{-}} = 0.01+x \tag{17}$$

#### 结合公式(15),(16),(17)我们会得到
$$lg\gamma = -0.51 \sqrt{0.01+x} \tag{18}$$

#### 同时，根据公式(1),(3),有
$$lg\gamma = 0.5 lg(\frac{K_{ap,AgBrO_{3}}}{x(0.01+x)}) \tag{19}$$

#### 此时我们就可以将(18),(19)结合，写成一个函数:$$f(x)=0.5lg(\frac{K_{ap,AgBrO_{3}}}{x(0.01+x)})+0.51 \sqrt{0.01+x}$$然后使用fsolve即可求解x的值

In [101]:
def func(x):
    Kap = 5.77E-5
    #return 0.5*(np.log10(Kap/(x*(0.01+x))))+(0.5115*((0.01+x)**0.5)/(1+((0.01+x))**0.5))
    return 0.5*(np.log10(Kap/(x*(0.01+x))))+(0.509*((0.01+x)**0.5))
root = fsolve(func,0.004)
print(root)

[0.00509747]


In [106]:
print("溶解度为"+str(solubility(molecular_weight_AgBrO3,root)))

溶解度为[1.20198431]


#### 有些书上会给出更详细的德拜休克耳公式$$lg\gamma = \frac{A |z_{+}z_{-}|\sqrt{I}}{1+B\dot{a}\sqrt{I}}\tag{13}$$
#### 这种公式考虑了离子半径等更详细的内容，其中B为常数,$\dot {a}$为参数，通常是中心离子与异电荷的平均距离。在低浓度下，这两个值通常均为1。如果你想要使用这个公式，只需要对上面的代码中计算gamma的代码进行改动即可

In [115]:
def calc_gamma(list_of_conc):
    
    conc_sum = sum(list_of_conc)
    return 10**(-0.51*(0.5*conc_sum)**0.5/(1+(0.5*conc_sum)**0.5))

In [120]:

solubility_list = [solubility_AgBrO3_in_water] 
solu_mol_AgBrO3_in_water = solubility_AgBrO3_in_water/molecular_weight_AgBrO3 #在这里，记得我们要把溶解度转化为溶解的物质的量!
conc_list = [0.01,solu_mol_AgBrO3_in_water,0.01+solu_mol_AgBrO3_in_water]
def calc_activ(Kap,tem_gamma,conc_list):
    b = conc_list[0]
    a = 1
    c = -Kap/(tem_gamma**2)
    return (-b+(b**2-4*a*c)**0.5)/2
for i in range(100): #通常对于这些简单方程，100次以内的迭代是充足的
    tem_gamma = calc_gamma(conc_list)
    tem_conc = calc_activ(Kap_AgBrO3,tem_gamma,conc_list)
    conc_list = [0.01,tem_conc,0.01+tem_conc]
    solubility_list.append(solubility(molecular_weight_AgBrO3,tem_conc))
    #print(tem_conc)
    if abs(solubility_list[-1]-solubility_list[-2]) < 0.0001: #根据我们预估的数值，0.0001的收敛限制应该属足够的
        break
print("此时溶解的浓度为"+str(tem_conc))
print("溶解度为"+str(solubility_list[-1]))

此时溶解的浓度为0.004977007568944842
溶解度为1.173578384757194
